# Project 3: Where's Waldo?

Due Mar. 4th

So far, we've mostly focused on using imagery to do stuff for which it is better suited than a human: calculating camera locations from imagery, finding an optimal projective transform to stitch images together, and (soon) we'll be doing "structure from motion" in which we create 3D models of the world from collections of 2D images.  These are tasks primarily based around measuring things and doing calculations.  On the other side of the coin is object recognition (identifying the semantic content of a scene), and the best contemporary computer vision algorithms do object recognition at roughly the level of a 2 year old human (with some exceptions).  For this (mini-)project, we're going to delve into a topic that sort of straddles the line between these two general realms of computer vision.

As a motivating example, did you ever play the game Where's Waldo.  There are books filled with images like the following:
<img src='waldo_1.jpg'>
The objective, of course, is to find Waldo, the man in the red striped shirt and beanie wearing glasses.  He looks like this:
<img src='waldo_template.jpg'>
These scenes are (obviously) intended to have a bunch of visual clutter to make this task reasonably challenging.

Your task will be to come up with an algorithm that locates the template image (Waldo's face) and the target image (the larger scene).  This is called *template matching*, and it's a primitive form of feature recognition.  

## Implementation
### Template Matching
Template matching works in a way that is very similar to filtering:  slide the template image over every location in the target image, computing some sort of metric at each position.  In practice, one commonly used choice for an error metric is the one that you've already used for matching keypoint descriptors: z-normalized sum square error.  Another choice is [normalized cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation#Normalization).  Once these metrics have been computed, simply find the argmin (for SSE) or argmax (for NCC), and this will be the location of the best match.  

**Your task is to implement template matching.  Use 'waldo_template.jpg' as the template and 'waldo_1.jpg' as the target image.  Where's Waldo? **

### Not so fast!!!  What about scale!
Oh, no.  As it turns out, the template I've provided is not the same scale as the Waldo in the image.  To deal with this, you'll need to create an image pyramid for the template (See Szeliski 3.5, and [Mubarak Shah's lecture on this topic](https://www.youtube.com/watch?v=KO7jJt0WHag&feature=youtu.be)).  This essentially just means creating a sequence of downsampled images of the template, and trying each one in hopes that one of the resulting down-scaled templates matches the feature in the target image.  **Create a sequence of templates with which to perform feature matching, each one 1/2 the resolution of the previous (so 1/4 the total number of pixels).  To avoid aliasing, before downsampling perform a $\sigma=1$ Gaussian Blur of the image.  Once you've built your image pyramid, find the argmin/max in 3 dimensions (u,v,template scale)**.

## Generalization
**Waldo appears in every Where's Waldo image (obviously).  Try using the same technique on 'waldo_2.jpg'.  Does the algorithm work?**  I confess that I pulled the image of waldo for the template directly from 'waldo_1.jpg', so for the correct scale, there is something close to an exact match (i.e. SSE=0).  However, Waldo, while easily recognizable to the human eye after undergoing the small scale deformations associated with artistic license, is not so easily recognizable via template matching.  We will return to a similar problem when discussing object recognition, and hopefully this example will motivate the need to come up with representations of objects (like Waldo) that are more robust.


In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
image = 'waldo_template.jpg'
image2 = 'waldo_1.jpg'
im = plt.imread(image2).mean(axis=2)

template = plt.imread(image).mean(axis=2)
plt.imshow(template, cmap='gray')

In [2]:
def gaussian(h_size, sigma):

    out = np.zeros((h_size, h_size))
    h_off = h_size // 2
    for j in range(h_size):
        for k in range(h_size):
            m, n = j - h_off, k - h_off
            out[j, k] = np.exp(-(m * m + n * n) / (2 * sigma * sigma))

    return out / out.sum()
    
def convolve(g, h):

    im_out = np.zeros_like(g)
    u, v = g.shape[0], g.shape[1]
    h_off = (h.shape[0] - 1) // 2
    

    for y in range(h_off, u - h_off):
        for x in range(h_off, v - h_off):
            im_sub = g[y - h_off:y + h_off + 1, x - h_off:x + h_off + 1]
            val = np.sum(im_sub * h)
            im_out[y, x] = val
            
    for i in range(h_off):
        im_out[i, :] = g[i, :] # first rows
        im_out[-(i+1), :] = g[-(i+1), :] # last rows
        im_out[:, i] = g[:, i] # first columns
        im_out[:, -(i+1)] = g[:, -(i+1)] # last columns

    return im_out

def z_norm(im):
        return (im - im.mean()) / np.std(im)

def error(im1, im2):
    return np.sum((im1 - im2) ** 2)

In [3]:
def pyramid_reduce(image, gaus_sigma):
    
    kernal = gaussian(5, gaus_sigma)
    im_gaus = convolve(image, kernal)
    
    reduced = np.zeros((im_gaus.shape[0]//2, im_gaus.shape[1]//2))
    for v in range(reduced.shape[0]):
        for u in range(reduced.shape[1]):
            reduced[v, u] = im_gaus[2*v + 1, 2*u + 1]
            
    return reduced

In [11]:
templ = plt.imread('waldo_template.jpg').mean(axis=2)
image = plt.imread('waldo_1.jpg').mean(axis=2)

u_out = []
v_out = []
scale_val = []
err_val = []

offs_v = templ.shape[0]//2
offs_u = templ.shape[1]//2
scale = 1
while scale >= 0.0315:
    print(templ.shape)
    er = np.inf
    u_val = 0
    v_val = 0
    templ_z = z_norm(templ)
    
    for v in range(offs_v, image.shape[0]-offs_v):
        for u in range(offs_u, image.shape[1]-offs_u):
            im_sub = image[v - offs_v:v + offs_v, u - offs_u:u + offs_u]
            im_z = z_norm(im_sub)
            err = error(templ_z, im_z)
            if err < er:
                er = err
                u_val = u
                v_val = v
                
    u_out.append(u_val)
    v_out.append(v_val)
    scale_val.append(scale)
    err_val.append(er)
    
    templ = pyramid_reduce(templ, 1)
    scale = scale * 0.5
    
out_array = np.column_stack((np.column_stack((np.column_stack((u_out, v_out)), scale_val)), err_val))    

(124, 100)


KeyboardInterrupt: 

In [7]:
out_array

array([[ 0.    ,  0.    ,  1.    ,     inf],
       [ 0.    ,  0.    ,  0.5   ,     inf],
       [ 0.    ,  0.    ,  0.25  ,     inf],
       [ 0.    ,  0.    ,  0.125 ,     inf],
       [ 0.    ,  0.    ,  0.0625,     inf]])